# git

In [ ]:
import ipywidgets as W, traitlets as T, pathlib as P, git as G, yaml, json, ipywidgets.embed as E
from tornado.ioloop import IOLoop
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor
from wxyz.dvcs import Watcher
from pathlib import Path

## Trackers
The nuts-and-bolts implementation of the bridge between `traitlets` and the on-disk representation.

### Base Tracker

In [ ]:
class _Tracker(W.Widget):
    executor = ThreadPoolExecutor(max_workers=1)
    tracked_widget = T.Instance(W.Widget)
    tracked_traits = W.trait_types.TypedTuple(T.Unicode(), allow_none=True, help="trait names to track (default all)")
    path = T.Instance(P.Path)
    encoding = T.Unicode("utf-8")
    change_number = T.Int(0)
    __extension__ = None
    
    async def on_user_change(self):
        raise NotImplementedError("tracker subclass must implement `async on_user_change`")
    
    async def on_file_change(self):
        raise NotImplementedError("tracker subclass must implement `async on_user_change`")
    
    def _on_user_change(self, change):
        try:
            IOLoop.current().add_callback(self.on_user_change)
            self.change_number += 1
        except:
            pass
        
    def _on_file_change(self, change):
        try:
            IOLoop.current().add_callback(self.on_file_change)
        except:
            pass

    @T.observe("tracked_widget")
    def _changed_tracked_widget(self, change):
        if change.old is not None and isinstance(change.old, W.Widget):
            change.old.unobserve(self._on_user_change, self.tracked_traits or T.All)
        if change.new is not None:
            change.new.observe(self._on_user_change, self.tracked_traits or T.All)
            self._on_file_change(None)
            
    @T.observe("tracked_traits")
    def _changed_tracked_traits(self, change):
        if self.tracked_widget is None or not isinstance(self.tracked_widget, W.Widget):
            return
        if change.old is None or isinstance(change.old, tuple):
            self.tracked_widget.unobserve(self._on_user_change, change.old or T.All)
        if change.new is None or isinstance(change.new, tuple):
            self.tracked_widget.observe(self._on_user_change, change.new or T.All)
            self._on_file_change(None)
    
    @classmethod
    def _detect_tracker(cls, path, base_cls=None):
        """ naive tracker finder... needs something cleverer
        """
        base_cls = base_cls or _Tracker
        
        if base_cls.__extension__ and path.name.endswith(base_cls.__extension__):
            return base_cls
        
        subclasses = base_cls.__subclasses__()
        
        for sub_cls in subclasses:
            if sub_cls.__extension__ and path.name.endswith(sub_cls.__extension__):
                return sub_cls
        
        for sub_cls in subclasses:
            subsub = cls._detect_tracker(path, sub_cls)
            if subsub:
                return subsub
        
        return None

### Dict Tracker

Store as a dictionary (aka key/value pair, map, hash), or generally vanilla JSON-compatible (no timestamps, etc).

In [ ]:
class _DictTracker(_Tracker):
    def dict_from_widget(self):
        d = {}
        w = self.tracked_widget
        if w is not None:
            s = E.dependency_state(w)[w.comm.comm_id]["state"]
            for k in self.tracked_traits or list(s):
                d[k] = s.get(k, None)
        return d
    
    def widget_from_dict(self, content):
         with self.tracked_widget.hold_trait_notifications():
            for trait in self.tracked_traits or self.tracked_widget.trait_names():
                if trait in content:
                    new_value = content[trait]
                    old_value = getattr(self.tracked_widget, trait) 
                    if new_value != old_value:
                        setattr(self.tracked_widget, trait, new_value)

### JSON Dict Tracker
While not the most naively revision-control-aware format, JSON is widely distributed.

In [ ]:
class JSONDictTracker(_DictTracker):
    __extension__ = ".json"
    
    @run_on_executor
    def on_user_change(self):
        self.path.write_text(
            json.dumps(self.dict_from_widget(), sort_keys=True, indent=True), 
            encoding=self.encoding
        )
    
    @run_on_executor
    def on_file_change(self):
        if not self.path.exists():
            return
        content = self.widget_from_dict(json.loads(self.path.read_text()))

### YAML Dict Tracker
YAML is somewhat more flexible for change control, but whitespace can be a bummer

In [ ]:
class YAMLDictTracker(_DictTracker):
    __extension__ = ".yaml"
    
    @run_on_executor
    def on_user_change(self):
        self.path.write_text(
            yaml.safe_dump(self.dict_from_widget(), default_flow_style=False), 
            encoding=self.encoding
        )
    
    @run_on_executor
    def on_file_change(self):
        if not self.path.exists():
            return

        self.widget_from_dict(yaml.safe_load(self.path.read_text()))

## BaseRepo
A base representation of a file-based repo. This would live in its own file, and not know anything about a specific DVCS. 

In [ ]:
class _Repo(W.Widget):
    working_dir = T.Instance(P.Path)
    watching = T.Bool(default_value=False)
    dirty = T.Bool(default_value=False)
    changes = T.Tuple(allow_none=True)
    head = T.Unicode()
    head_history = T.Tuple()
    heads = W.trait_types.TypedTuple(T.Unicode(), default_value=tuple())
    _watcher = T.Instance(Watcher, allow_none=True)
    _trackers = W.trait_types.TypedTuple(T.Instance(_Tracker), default_value=tuple())
    _change_link = None
    
    def __init__(self, working_dir, *args, **kwargs):
        kwargs["working_dir"] = P.Path(kwargs.get("working_dir", working_dir))
        super().__init__(*args, **kwargs)
    
    @T.default("changes")
    def _default_changes(self):
        return tuple()
    
    @T.observe("watching", "path")
    def _on_watching(self, change):
        if self._watcher:
            if self._change_link:
                self._change_link.unlink()
                self._change_link = None
            self._watcher.watching = False
            self._watcher = None
        if self.watching:
            self._watcher = Watcher(self.working_dir)
            self._change_link = T.dlink((self._watcher, "changes"), (self, "changes"), self._on_watch_changes)
            self._watcher.watching = True
    
    def _on_watch_changes(self, changes):
        """ react to changes from the watcher
        """
        self.log.warn("changes %s", changes)
        return changes
    
    def track(self, **kwargs):
        """ create a tracker for the given widget
        """
        assert self.working_dir.exists()
        kwargs["path"] = P.Path(self.working_dir / kwargs["path"])
        tracker_cls = kwargs.pop("tracker_cls", None)
        if tracker_cls is None:
            tracker_cls = _Tracker._detect_tracker(kwargs["path"])
        assert tracker_cls is not None
        tracker = tracker_cls(**kwargs)
        self._trackers += tracker,
        return tracker

## git

Specialization for supporting `git`.

In [ ]:
class Repo(_Repo):
    _git = T.Instance(G.Repo, allow_none=True)
    
    @T.observe("working_dir")
    def _on_path(self, change):
        if change.new:
            self._git = G.Repo.init(change.new)
            ignore = Path(change.new) / ".gitignore"
            if not ignore.exists():
                ignore.write_text(".ipynb_checkpoints/")
                self.commit("initial commit")
            else:
                self._update_head_history()
    
    @T.default("head")
    def _default_head(self):
        return self._git.head.name
    
    @T.observe("head")
    def _on_head_changed(self, change):
        if change.new:
            self._update_head_history()
    
    def _update_head_history(self):
        try:
            head = [h for h in self._git.heads if h.name == self.head][0]
            self.head_history = [
                {
                    "commit": c.newhexsha, 
                    "timestamp": c.time[0],
                    "message": c.message,
                    "author": {
                        "name": c.actor.name,
                        "email": c.actor.email
                    }
                }
                for c in head.log()[::-1]
            ]
        except Exception as err:
            self.log.warn(err)
            self.head_history = []
    
    def _update_heads(self):
        """ TODO: when does this get updated?
        """
        self.heads = [head.name for head in self._git.heads]
        if self.head in [None, "HEAD"] and self.heads:
            self.head = "master"

        self._update_head_history()

    def _on_watch_changes(self, *changes):
        self.dirty = self._git.is_dirty()
        if self._watcher:
            for change in self._watcher.changes:
                for tracker in self._trackers:
                    tracked_path = Path(self._git.working_dir) / change["path"]
                    if tracker.path.resolve() == tracked_path.resolve():
                        tracker._on_file_change(None)
        return [
            dict(
                a_path=diff.a_path,
                b_path=diff.b_path,
                change_type=diff.change_type
            )
            for diff in self._git.index.diff(None)
        ] + [
            dict(
                a_path=None,
                b_path=ut,
                change_type="U"
            ) for ut in self._git.untracked_files
        ]
    
    def stage(self, path):
        self._git.index.add(path)
    
    def unstage(self, path):
        self._git.index.remove(path)
    
    def commit(self, message):
        self._git.index.commit(message)
        self._on_watching(None)
        self._update_heads()
        
    def revert(self, ref):
        self._git.head.commit = ref
        self._git.head.reset(index=True, working_tree=True)
        self._update_heads()